In [1]:
import json

file_path = "/home/simon/Documents/data_science/Thesis/micha_gold_truth/Qwen_Qwen3-235B-A22B-Instruct-2507-FP8_vllm__benchmark_table_template_filling__temperature_0__top_3_rag_examples__loop_0__no_think_queued.json"

with open(file_path, "r") as f:
    content = f.read().replace('"NaN"', 'null').replace('NaN', 'null')
    data = json.loads(content)

In [23]:
import pandas as pd

for entry in data:
    filepath = entry.get("filepath", "").replace("/pvc/benchmark_truth/real_tables/", "").replace("__", "/").replace(".pdf", "")
    df_string = entry.get("result", {})
    df = pd.read_json(df_string)
    # Save to Excel with formatting
    # Split filepath by "/" and join back with "/" after adding prefix to the last part
    parts = filepath.split("/")
    parts[-1] = "aktiva_" + parts[-1]
    final_path = "/".join(parts)
    try:
        with pd.ExcelWriter(f"/home/simon/Documents/data_science/Thesis/micha_gold_truth/{final_path}.xlsx", engine='xlsxwriter') as writer:
            df.to_excel(writer, index=False)
            workbook  = writer.book
            worksheet = writer.sheets['Sheet1']

            # Format for monospace and float with 2 decimals
            mono_float_format = workbook.add_format({'font_name': 'Courier New', 'num_format': '0.00'})
            # Format for wrapped text
            wrap_format = workbook.add_format({'text_wrap': True})
            # Format for gray background
            gray_bg_format = workbook.add_format({'bg_color': '#D3D3D3'})

            # Set column widths and formats
            for i, col in enumerate(df.columns):
                if col in ['year', 'previous_year']:
                    worksheet.set_column(i, i, 13, mono_float_format)
                else:
                    worksheet.set_column(i, i, 36, wrap_format)

            # Apply wrap format to all cells in first three columns
            for row in range(1, len(df) + 1):
                for col_idx in range(3):
                    worksheet.write(row, col_idx, df.iloc[row - 1, col_idx], wrap_format)

            # Apply gray background to rows with no values in year and previous_year
            for row in range(1, len(df) + 1):
                if pd.isna(df.iloc[row - 1]['year']) and pd.isna(df.iloc[row - 1]['previous_year']):
                    worksheet.set_row(row, None, gray_bg_format)
    except Exception as e:
        print(f"Skipping file {final_path} due to error: {e}")

/tmp/ipykernel_417195/4037687940.py:6: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(df_string)
/tmp/ipykernel_417195/4037687940.py:6: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(df_string)
/tmp/ipykernel_417195/4037687940.py:6: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(df_string)
/tmp/ipykernel_417195/4037687940.py:6: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(df_string)
/tmp/ipykernel_417195/4037687940.py:6: Futur

Skipping file Tempelhof Projekt GmbH /230829_THF_Gescha/aktiva_ftsbericht_2022 due to error: Cannot save file into a non-existent directory: '/home/simon/Documents/data_science/Thesis/micha_gold_truth/Tempelhof Projekt GmbH /230829_THF_Gescha'


/tmp/ipykernel_417195/4037687940.py:6: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(df_string)
/tmp/ipykernel_417195/4037687940.py:6: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(df_string)
/tmp/ipykernel_417195/4037687940.py:6: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(df_string)
/tmp/ipykernel_417195/4037687940.py:6: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(df_string)
/tmp/ipykernel_417195/4037687940.py:6: Futur

In [4]:
data[0]

{'filepath': '/pvc/benchmark_truth/real_tables/Amt für Statistik Berlin-Brandenburg__AP_Geschaeftsbericht_DE_2016_BBB.pdf',
 'result': '[{"E1":"Anlagevermoegen","E2":"Immaterielle Vermoegensgegenstaende","E3":"Selbst geschaffene gewerbliche Schutzrechte und aehnliche Rechte und Werte","year":null,"previous_year":null},{"E1":"Anlagevermoegen","E2":"Immaterielle Vermoegensgegenstaende","E3":"Geschaefts- oder Firmenwert","year":null,"previous_year":null},{"E1":"Anlagevermoegen","E2":"Immaterielle Vermoegensgegenstaende","E3":"geleistete Anzahlungen","year":null,"previous_year":null},{"E1":"Anlagevermoegen","E2":"Immaterielle Vermoegensgegenstaende","E3":"entgeltlich erworbene Konzessionen, gewerbliche Schutzrechte und aehnliche Rechte und Werte sowie Lizenzen an solchen Rechten und Werten","year":105541.0,"previous_year":144713.0},{"E1":"Anlagevermoegen","E2":"Sachanlagen","E3":"Grundstuecke, grundstuecksgleiche Rechte und Bauten einschliesslich der Bauten auf fremden Grundstuecken","year

In [25]:
import os
import shutil

src_dir = "/home/simon/Documents/data_science/Thesis/micha_gold_truth"
dst_dir = "/home/simon/Documents/data_science/Thesis/micha_gold_truth_aktiva_only"

for root, dirs, files in os.walk(src_dir):
    # Compute relative path from src_dir
    rel_path = os.path.relpath(root, src_dir)
    # Compute destination directory path
    dst_subdir = os.path.join(dst_dir, rel_path)
    # Create destination subdirectory if it doesn't exist
    os.makedirs(dst_subdir, exist_ok=True)
    for file in files:
        if file.startswith("aktiva_") or file.endswith(".xlsx"):
            src_file = os.path.join(root, file)
            dst_file = os.path.join(dst_subdir, file)
            shutil.copy2(src_file, dst_file)

In [1]:
import pandas as pd

In [10]:
df = pd.read_csv("/home/simon/Documents/data_science/Thesis/micha_gold_truth_aktiva_only_KORRIGIERT/pdf_files_list_with_pages.csv")
df

,pdf_path,page_aktiva,checked,errors,comment
0,/home/simon/Downloads/micha_gold_truth/Amt für...,32,x,0,NaN
1,/home/simon/Downloads/micha_gold_truth/Amt für...,32,x,0,NaN
2,/home/simon/Downloads/micha_gold_truth/Amt für...,28,x,0,NaN
3,/home/simon/Downloads/micha_gold_truth/Amt für...,32,x,0,NaN
4,/home/simon/Downloads/micha_gold_truth/Amt für...,8,x,0,NaN
...,...,...,...,...,...
102,/home/simon/Downloads/micha_gold_truth/WISTA-M...,1,x,0,NaN
103,/home/simon/Downloads/micha_gold_truth/Zoologi...,66,x,4,grüne Werte standen in roter Zeile
104,/home/simon/Downloads/micha_gold_truth/Zoologi...,117,x,0,NaN
105,/home/simon/Downloads/micha_gold_truth/Zoologi...,128,x,0,NaN


In [30]:
# Convert df_table_characterists_goal to a DataFrame to add new columns
df_table_characterists_goal = pd.DataFrame(df['pdf_path'].copy())
df_table_characterists_goal['company'] = df['pdf_path'].apply(lambda x: x.split("/")[-2])
df_table_characterists_goal['filename'] = df['pdf_path'].apply(lambda x: x.split("/")[-1])
df_table_characterists_goal.drop(columns=['pdf_path'], inplace=True)

df_table_characterists_goal

,company,filename
0,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2016_BBB.pdf
1,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2017_BBB.pdf
2,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2018_BBB.pdf
3,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2019_BBB.pdf
4,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2020_BBB.pdf
...,...,...
102,WISTA-Management GmbH,2021_WISTA_Bilanz.pdf
103,Zoologischer Garten Berlin AG,Geschaeftsbericht_AG_2021.pdf
104,Zoologischer Garten Berlin AG,Geschaeftsbericht_AG_2022.pdf
105,Zoologischer Garten Berlin AG,Geschaeftsbericht_AG_2023.pdf


In [31]:
df_table_characterists = pd.read_csv("/home/simon/Documents/data_science/Thesis/benchmark_truth/real_tables/table_characteristics.csv")
df_table_characterists

,company,filename,T_in_year,T_in_previous_year,sum_same_line,spacer,n_columns,vorjahr,passiva_same_page,header_span,enumeration,vis_seperated_cols,vis_seperated_rows,sum_in_header,year_missing,geschaeftsjahr
0,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2016_BBB.pdf,0,0,0,,3,1,0,1,1,0,0,0,0,0
1,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2017_BBB.pdf,0,0,0,,3,1,0,1,1,0,0,0,0,0
2,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2018_BBB.pdf,0,0,0,,3,1,0,1,1,0,0,0,0,0
3,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2019_BBB.pdf,0,0,0,,3,0,0,1,1,0,0,0,0,0
4,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2020_BBB.pdf,0,0,0,,3,0,0,1,1,0,0,0,0,0
5,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2021_BBB.pdf,0,0,0,,3,0,0,1,1,0,0,0,0,0
6,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2022_BBB.pdf,0,0,0,,3,0,0,1,1,0,0,0,0,0
7,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2023_BBB.pdf,0,0,0,.,3,0,0,1,1,0,0,0,0,0
8,Berliner Bäder Betriebe,2018_Geschaeftsbericht_BBB.pdf,0,0,0,.,3,0,0,1,1,0,0,0,0,0
9,Berliner Stadtgüter,BSG-Geschaeftsbericht-2021-druck.pdf,0,0,0,.,3,0,0,1,1,0,0,0,0,0


In [32]:
df_table_characterists_goal = df_table_characterists_goal.merge(df_table_characterists, on=['company', 'filename'], how='left')
df_table_characterists_goal

,company,filename,T_in_year,T_in_previous_year,sum_same_line,spacer,n_columns,vorjahr,passiva_same_page,header_span,enumeration,vis_seperated_cols,vis_seperated_rows,sum_in_header,year_missing,geschaeftsjahr
0,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2016_BBB.pdf,0.0,0.0,0.0,,3.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
1,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2017_BBB.pdf,0.0,0.0,0.0,,3.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2018_BBB.pdf,0.0,0.0,0.0,,3.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2019_BBB.pdf,0.0,0.0,0.0,,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
4,Amt für Statistik Berlin-Brandenburg,AP_Geschaeftsbericht_DE_2020_BBB.pdf,0.0,0.0,0.0,,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,WISTA-Management GmbH,2021_WISTA_Bilanz.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,Zoologischer Garten Berlin AG,Geschaeftsbericht_AG_2021.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,Zoologischer Garten Berlin AG,Geschaeftsbericht_AG_2022.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
105,Zoologischer Garten Berlin AG,Geschaeftsbericht_AG_2023.pdf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df_table_characterists_goal.to_csv("/home/simon/Documents/data_science/Thesis/benchmark_truth/real_tables/table_characteristics_more_examples.csv", index=False)